# Llama on LOCAL

## Example 1  - LLAMA3 using Ollama

The project is about caracterizing transactions and generating a dashboard. 

- The original author __Thu Vu data analytics__ publications/data can be found here: 

https://www.youtube.com/watch?v=h_GTxRFYETY

https://github.com/thu-vu92/local-llms-analyse-finance

https://ollama.com/


This example includes: 

- LLM Categorization of transactions 

- DASHBOARD on Ploty plot 

### Getting Ollama: 

1. Go to ollama.ai 

2. Click download 

### Installing Ollama

1. Double clink on ollama downloaded file and follow instructions 

### Get models

1. on Terminal: 

    ollama pull llama3

3. Generate own model (Optional), create a file using nano: 

```nano expense_analyzer```

Enter this information in the file and save it: 

        FROM llama3

        PARAMETER temperature 0.8

        SYSTEM You are a financial assistan, you help classify expenses and income from bank trnsactions

3. Run this code

```ollama create expense_analyzer_llama3 -f ./expense_analyzer```

4. To remov the model: 

```ollama rm <model_name>```


In [ ]:
from langchain_community.llms import Ollama
import pandas as pd
# Generate the model
llm3 = Ollama(model ='llama3')

# This is testing the connetion to the model works: 
llm3.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
ISS Catering Services De Meern, Vishandel Sier AMSTELVEEN, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

In [ ]:
# Uploading the dataset with the transactions 
df = pd.read_csv('transactions_2022_2023.csv', low_memory = False )
print(df.head())
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
print('\nNumber of unique transactions: ', len(unique_transactions))
unique_transactions[1:10]

In [ ]:
# Get index list
# https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop
# Spit the index into groups of 30
index_list = list(hop(0, len(unique_transactions), 30))
index_list

In [ ]:
# Output validation using pydanthic
from pydantic import BaseModel, field_validator
from typing import List

# Validate response format - check if it actually contains hyphen ("-")
class ResponseChecks(BaseModel):
    data: List[str]

    @field_validator("data")
    def check(cls, value):
        for item in value:
            if len(item) > 0:
                assert "-" in item, "String does not contain hyphen."

# Test validation
ResponseChecks(data=['Hello - World', 'Hello - there!'])

In [ ]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')
    # Keep only the lines in between blank lines (removing the explaination lines at the beginning and end of the response)
    blank_indexes = [index for index in range(len(response)) if response[index] == '']
    if len(blank_indexes) == 1:
        response = response[(blank_indexes[0] + 1):]
    else:
        response = response[(blank_indexes[0] + 1) : blank_indexes[1]]
    # Print response and validate if it is in the correct format
    print(response)
    #### Was getting an error and include this step to prevent it to happen
    response = [s.replace('* ', '') for s in response] 
    # return response
    ResponseChecks(data = response)    
    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    return categories_df
    #### Keeping the below inside the function did not work. I move this outside after copying the dataframe into a new one. 
    # categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    # return categories_df

In [ ]:
# Test out the function
r = categorize_transactions('ISS Catering Services De Meern, Vishandel Sier AMSTELVEEN, Etos AMSTERDAM NLD, Bistro Bar Amsterdam', llm3)
r[['Transaction', 'Category']] = r['Transaction vs category'].str.split(' - ', expand=True)
r

In [ ]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()
max_tries = 7

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    # Try and validate output, if it fails, try again for max_tries=7 times
    for j in range(1, max_tries):
        try:
            categories_df = categorize_transactions(transaction_names, llm3)
            categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)
            
        except:
            if j < max_tries:
                continue
            else:
                raise Exception(f"Cannot categorise transactions indexes {i} to {i+1}.")
        break
categories_df.head()

In [ ]:
categories_df_all[['Transaction', 'Category']] = categories_df_all['Transaction vs category'].str.split(' - ', expand=True)
print(categories_df_all.head(5))
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

In [ ]:
### Manual formating of some categories
# Drop NA values
categories_df_all = categories_df_all.dropna()
# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"



# Remove the numbering eg "1. " from Transaction column
# This did not worked
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')  
categories_df_all


In [ ]:
### Making a copy of the dataframe
cat_copy = categories_df_all.copy()
# Removing the numbers in the category
cat_copy.loc[:,'Transact'] = cat_copy['Transaction'].str.split('.', expand=True)[1]
cat_copy.loc[:,'Transact'] = cat_copy.Transact.str.strip()
cat_copy.drop(['Transaction', 'Transaction vs category'], axis = 1, inplace = True)
cat_copy.rename(columns = {'Transact': 'Transaction'}, inplace = True)
cat_copy.Transaction.unique()

In [ ]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = df.merge(cat_copy, left_on='Name / Description', right_on='Transaction', how='left')
df

In [ ]:
categorizefilenm = "transactions_2022_2023_categorized.csv"
df.to_csv(categorizefilenm, index=False)

In [ ]:
### Importing libraries for dashboard. 
import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn

if df.empty:    
    # Read transactions_2022_2023_categorized.csv
    df = pd.read_csv(categorizefilenm = "transactions_2022_2023_categorized.csv", low_memory = False)

# Add year and month columns
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Month Name'] = pd.to_datetime(df['Date']).dt.strftime("%b")
# Remove "Transaction" and "Transaction vs category" columns
df = df.drop(columns=['Transaction'])
# For Income rows, assign Name / Description to Category
df['Category'] = np.where(df['Expense/Income'] == 'Income', df['Name / Description'], df['Category'])

In [ ]:
def make_pie_chart(df, year, label):
    # Filter the dataset for expense transactions
    sub_df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]

    color_scale = px.colors.qualitative.Set2
    
    pie_fig = px.pie(sub_df, values='Amount (EUR)', names='Category', color_discrete_sequence = color_scale)
    pie_fig.update_traces(textposition='inside', direction ='clockwise', hole=0.3, textinfo="label+percent")

    total_expense = df[(df['Expense/Income'] == 'Expense') & (df['Year'] == year)]['Amount (EUR)'].sum() 
    total_income = df[(df['Expense/Income'] == 'Income') & (df['Year'] == year)]['Amount (EUR)'].sum()
    
    if label == 'Expense':
        total_text = "€ " + str(round(total_expense))

        # Saving rate:
        saving_rate = round((total_income - total_expense)/total_income*100)
        saving_rate_text = ": Saving rate " + str(saving_rate) + "%"
    else:
        saving_rate_text = ""
        total_text = "€ " + str(round(total_income))

    pie_fig.update_layout(uniformtext_minsize=10, 
                        uniformtext_mode='hide',
                        title=dict(text=label+" Breakdown " + str(year) + saving_rate_text),
                        # Add annotations in the center of the donut.
                        annotations=[
                            dict(
                                text=total_text, 
                                # Square unit grid starting at bottom left of page
                                x=0.5, y=0.5, font_size=12,
                                # Hide the arrow that points to the [x,y] coordinate
                                showarrow=False
                            )
                        ]
                    )
    return pie_fig
income_pie_fig_2022 = make_pie_chart(df, 2022, 'Income')
income_pie_fig_2022

In [ ]:
def make_monthly_bar_chart(df, year, label):
    df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]
    total_by_month = (df.groupby(['Month', 'Month Name'])['Amount (EUR)'].sum()
                        .to_frame()
                        .reset_index()
                        .sort_values(by='Month')  
                        .reset_index(drop=True))
    if label == "Income":
        color_scale = px.colors.sequential.YlGn
    if label == "Expense":
        color_scale = px.colors.sequential.OrRd
    
    bar_fig = px.bar(total_by_month, x='Month Name', y='Amount (EUR)', text_auto='.2s', title=label+" per month", color='Amount (EUR)', color_continuous_scale=color_scale)
    # bar_fig.update_traces(marker_color='lightslategrey')
    
    return bar_fig
income_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Income')
income_monthly_2022

In [ ]:
# Pie charts
income_pie_fig_2022 = make_pie_chart(df, 2022, 'Income')
expense_pie_fig_2022 = make_pie_chart(df, 2022, 'Expense')  
income_pie_fig_2023 = make_pie_chart(df, 2023, 'Income')
expense_pie_fig_2023 = make_pie_chart(df, 2023, 'Expense')

# Bar charts
income_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Income')
expense_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Expense')
income_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Income')
expense_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Expense')

# Create tabs
tabs = pn.Tabs(
                        ('2022', pn.Column(pn.Row(income_pie_fig_2022, expense_pie_fig_2022),
                                                pn.Row(income_monthly_2022, expense_monthly_2022))),
                        ('2023', pn.Column(pn.Row(income_pie_fig_2023, expense_pie_fig_2023),
                                                pn.Row(income_monthly_2023, expense_monthly_2023))
                        )
                )
tabs.show()

In [ ]:
# Dashboard template
template = pn.template.FastListTemplate(
    title='Personal Finance Dashboard',
    sidebar=[pn.pane.Markdown("# Income Expense analysis"), 
             pn.pane.Markdown("Overview of income and expense based on my bank transactions. Categories are obtained using local LLMs."),
             pn.pane.PNG("picture.png", sizing_mode="scale_both")
             ],
    main=[pn.Row(pn.Column(pn.Row(tabs)
                           )
                ),
                ],
    # accent_base_color="#88d8b0",
    header_background="#c0b9dd",
)

template.show()

____
## Example 2  - LLAMA3 using LancChain

The project is about caracterizing transactions and generating a dashboard. 

- The original author __LanngChain__ publications/data can be found here: 

https://www.youtube.com/watch?v=-ROS6gfYIts

https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_rag_agent_llama3_local.ipynb

https://python.langchain.com/v0.2/docs/integrations/llms/ollama/





In [ ]:
### For Tracing. 
### Create an account @ https://smith.langchain.com/
### Generate an API Key in the settings menu


import sys, os
# import google.generativeai as genai
sys.path.append(os.path.abspath(os.path.join('../', 'secret')))
from secret_info import lanchain_trace_api

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = lanchain_trace_api

In [ ]:
from langchain_community.llms import Ollama
# assuming you have Ollama installed and have llama3 model pulled with `ollama pull llama3 `
llm = Ollama(model="llama3")  
llm.invoke("Tell me a joke")

In [ ]:
#%pip install --upgrade --quiet  langchain langchain-community langchainhub gpt4all langchain-chroma 
# gpt4all_embd = GPT4AllEmbeddings()

# %pip install --upgrade --quiet  gpt4all > /dev/null
# !pip install chromadb
# %pip install -U langchain-nom?ic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all langchain-text-splitters

!pip install sentence-transformers

In [4]:
### LLM - Name of the model
local_llm = "llama3"
### Index

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import OllamaEmbeddings

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# In order to work needs Python 3.10
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
#    embedding = HuggingFaceEmbeddings(),
#    persist_directory= False
    embedding=OllamaEmbeddings(model="llama3"), #GPT4AllEmbeddings(),
)
retriever = vectorstore.as_retriever()


KeyboardInterrupt: 

In [ ]:
from gpt4all import Embed4All
text = 'The quick brown fox jumps over the lazy dog'
embedder = Embed4All()
output = embedder.embed(text)
print(output)

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
# ollama_emb = OllamaEmbeddings(
#     model="llama3",
# )

In [ ]:
from langchain.vectorstores.chroma import Chroma


db = Chroma.from_documents( documents=doc_splits,
    collection_name="rag-chroma",
    embedding = GPT4AllEmbeddings())


In [ ]:
!pip install gpt4all

In [ ]:
# Embed and index
run_local = 'Yes'
if run_local == "Yes":
    embedding = GPT4AllEmbeddings()
else:
    embedding = MistralAIEmbeddings(mistral_api_key=mistral_api_key)

In [ ]:
import os 
script_dir = os.path.dirname(os.path.realpath('.'))
model_path = os.path.join(script_dir, "all-MiniLM-L6-v2.gguf2.f16.gguf")
model_path

In [6]:
### Retrieval Grader

from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance 
    of a retrieved document to a user question. If the document contains keywords related to the user question, 
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()
question = "agent memory"
docs = llm.invoke(question)
doc_txt = docs[1].page_content
# print(retrieval_grader.invoke({"question": question, "document": doc_txt}))



TypeError: 'AIMessage' object is not subscriptable

In [7]:
docs

AIMessage(content='{} ', response_metadata={'model': 'llama3', 'created_at': '2024-06-03T00:01:52.805705Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1366762878, 'load_duration': 6466865, 'prompt_eval_count': 11, 'prompt_eval_duration': 884978000, 'eval_count': 3, 'eval_duration': 467890000}, id='run-cc125902-a964-42a4-9541-37d67a8d70e1-0')

In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())

In [ ]:
# gpt4all_embd = GPT4AllEmbeddings()
%pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all langchain-text-splitters


In [ ]:
%pip install langchain sentence_transformers


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()
text = "This is a test document."
query_result = embeddings.embed_query(text)
print(query_result [:3])

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub gpt4all langchain-chroma 

In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())

In [ ]:
Chroma??